In [49]:
import requests

In [50]:
dtm_url = "https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/dgm1_xyz/"

In [51]:
r = requests.get(dtm_url)

In [52]:
r.content[:2000]

b'<?xml version="1.0" encoding="UTF-8" standalone="yes" ?>\n<opengeodata>\n\t<metafiles>\n\t\t<metafile name="dgm1_meta.zip" size="111037" timestamp="2021-10-05T22:20:04.486715" />\n\t</metafiles>\n\n\t<datasets>\n\t\t<dataset name="dgm1_kacheln" title="Digitales Gel\xc3\xa4ndemodell - Gitterweite 1 m (XYZ) - Paketierung: Einzelkacheln"><files>\n\t\t\t\t<file name="dgm1_32_280_5652_1_nw.xyz.gz" size="3232513" timestamp="2020-06-18T11:23:24" />\n\t\t\t\t<file name="dgm1_32_280_5653_1_nw.xyz.gz" size="3218343" timestamp="2020-06-18T11:43:07" />\n\t\t\t\t<file name="dgm1_32_280_5654_1_nw.xyz.gz" size="2955431" timestamp="2020-06-18T11:50:47" />\n\t\t\t\t<file name="dgm1_32_280_5655_1_nw.xyz.gz" size="2867917" timestamp="2020-06-18T11:51:17" />\n\t\t\t\t<file name="dgm1_32_280_5656_1_nw.xyz.gz" size="2803718" timestamp="2020-06-18T11:51:36" />\n\t\t\t\t<file name="dgm1_32_280_5657_1_nw.xyz.gz" size="2825547" timestamp="2020-12-02T11:05:23" />\n\t\t\t\t<file name="dgm1_32_280_5658_1_nw.xyz.

In [53]:
conda install -c conda-forge bs4

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [54]:
from bs4 import BeautifulSoup 

In [55]:
soup = BeautifulSoup(r.content, 'html.parser')

In [56]:
tag_list = soup.find_all('file')

In [57]:
len(tag_list)

35860

In [58]:
tag = tag_list[3]
print(type(tag))
print(tag)

<class 'bs4.element.Tag'>
<file name="dgm1_32_280_5655_1_nw.xyz.gz" size="2867917" timestamp="2020-06-18T11:51:17"></file>


In [59]:
tag["name"]

'dgm1_32_280_5655_1_nw.xyz.gz'

In [60]:
fname_list = []

for tag in tag_list:
    fname_list.append(tag["name"])


In [61]:
fname_list = [tag["name"] for tag in tag_list]

In [62]:
fname_list[:10]

['dgm1_32_280_5652_1_nw.xyz.gz',
 'dgm1_32_280_5653_1_nw.xyz.gz',
 'dgm1_32_280_5654_1_nw.xyz.gz',
 'dgm1_32_280_5655_1_nw.xyz.gz',
 'dgm1_32_280_5656_1_nw.xyz.gz',
 'dgm1_32_280_5657_1_nw.xyz.gz',
 'dgm1_32_280_5658_1_nw.xyz.gz',
 'dgm1_32_280_5659_1_nw.xyz.gz',
 'dgm1_32_280_5660_1_nw.xyz.gz',
 'dgm1_32_280_5661_1_nw.xyz.gz']

In [63]:
fname_list[3].split("_")[2]

'280'

In [64]:
xs,ys = fname_list[3].split("_")[2:4]
print(f"xs:{xs+'000':s} , ys:{ys +'000':s}")

xs:280000 , ys:5655000


In [65]:
Dx = Dy = 1000

In [66]:
x_L = int(fname_list[3].split("_")[2])*1000
y_L = int(fname_list[3].split("_")[3])*1000
x_R = x_L + Dx
y_U = y_L + Dy

In [67]:
P_LL = (x_L,y_L)
P_UL = (x_L,y_U)
P_UR = (x_R,y_U)
P_LR = (x_R,y_L)

In [68]:
def points_from_fname(fname,Dx=1000,Dy=1000):
    x_L = int(fname_list[3].split("_")[2])*1000
    y_L = int(fname_list[3].split("_")[3])*1000
    x_R = x_L + Dx
    y_U = y_L + Dy
    
    P_LL = (x_L,y_L)
    P_UL = (x_L,y_U)
    P_UR = (x_R,y_U)
    P_LR = (x_R,y_L)
    
    return[P_LL,P_UL,P_UR,P_LR]

In [69]:
points_from_fname(fname_list[1])

[(280000, 5655000), (280000, 5656000), (281000, 5656000), (281000, 5655000)]

In [70]:
#for fname in fname_list:
 #   print(points_from_fname(fname))

In [71]:
from shapely.geometry import Polygon 
geometry = Polygon( points_from_fname(fname_list))


In [72]:
import geopandas 
import geopandas as gpd

In [73]:
geom_list = [Polygon(points_from_fname(fname)) for fname in fname_list]

In [74]:
dic = {'Name' : fname_list, 'geometry' : geom_list} 

In [75]:
gdf = gpd.GeoDataFrame(data = dic, crs = "EPSG:25832")


In [76]:
gdf.head()

,Name,geometry
0,dgm1_32_280_5652_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
1,dgm1_32_280_5653_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
2,dgm1_32_280_5654_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
3,dgm1_32_280_5655_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
4,dgm1_32_280_5656_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."


In [77]:
data_dir = r"../data/derived/"
out_fname = r"NRW_DTM_Tiles_BB.gpkg"
print(f"Write gdf to file {data_dir + out_fname:s}")
gdf.to_file(data_dir + out_fname, driver = "GPKG") 

Write gdf to file ../data/derived/NRW_DTM_Tiles_BB.gpkg


In [78]:
print(f"gdf.shape: {gdf.shape}")
gdf.head()

gdf.shape: (35860, 2)


,Name,geometry
0,dgm1_32_280_5652_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
1,dgm1_32_280_5653_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
2,dgm1_32_280_5654_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
3,dgm1_32_280_5655_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
4,dgm1_32_280_5656_1_nw.xyz.gz,"POLYGON ((280000.000 5655000.000, 280000.000 5..."
